In [1]:
from scipy import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown, RadioButtons


#plt.style.use(['science', 'no-latex', 'high-vis'])
plt.rc('font', family='Arial')
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)

plt.rc('axes', labelsize=18, titlesize=16)

plt.rcParams["ytick.major.size"] = 10
plt.rcParams["ytick.minor.size"] = 5

In [2]:
z = np.arange(0,51,1)

y_rad = np.loadtxt('y_rad.dat').reshape(-1,1)

h_rad = [np.loadtxt(f'h{i+1}_rad.dat') for i in range(10)]
h_rad = np.array(h_rad)

x_rad = np.loadtxt('x_rad.dat').reshape(-1,1)
xb_rad1 = np.loadtxt('xb_rad1.dat').reshape(-1,1)
xb_rad2 = np.loadtxt('xb_rad2.dat').reshape(-1,1)



$
\hat x_a = (H^T H)^{-1} R^{-1} y
$

In [11]:
# Case 4

def plot_T(ep):

    Eii = ep*y_rad/100
    R = np.diag(Eii.squeeze()**2)

    x_rad1 = ((np.linalg.inv(h_rad.T @ h_rad) @ h_rad.T) @ np.linalg.inv(R)) @ y_rad

    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(121)

    ax.plot(x_rad,z,lw=2, label='True', c='b')
    ax.plot(x_rad1,z,ls='-', label='Retrieval', c='r', marker='o', lw=0.8)

    ax.set_xlabel('$T$ (K)',size=20)
    ax.set_ylabel('$z$ (km)',size=20)
    ax.set_title(f'$e_R$ = {ep:.0f}%')
    ax.set_xlim(180,300)
    ax.legend(prop=dict(size=20))


    ax = fig.add_subplot(122)
    ax.plot(x_rad1-x_rad,z,c='k')
    ax.set_xlim(-50,50)
    ax.set_xlabel('$T$ bias (K)', size=20)
    ax.set_title(f'$e_R$ = {ep:.0f}%')
    
interact(plot_T, ep=FloatSlider(min=1, max=50, step=1, value=5))

interactive(children=(FloatSlider(value=5.0, description='ep', max=50.0, min=1.0, step=1.0), Output()), _dom_c…

<function __main__.plot_T(ep)>

$
K = P_b H^T (R + HP_bH^T)^{-1} \\
A = KH \\
\hat x_a = x_b + K(y - Hx_b)
$


In [19]:
def plot_T(ep,sigmab,L,xb):
    #sigmab = 50
    #L = 3

    ep1 = ep*y_rad/100
    R = np.diag(ep1.squeeze()**2)

    Pb = np.full((51,51),0)

    for i in range(51):
        Pb[i,i] = sigmab**2
        #print(i,i,end=' , ')
        for j in range(i):
            #print(i,j)
            Pb[i,j] = (Pb[i,i]*Pb[j,j])**0.5 * np.exp(-((z[i] - z[j])**2/(L**2)))
            #Pb[i,j] = sigmab**2 * np.exp(-((z[i] - z[j])**2/(L**2)))
            Pb[j,i] = Pb[i,j]
        
    K = (Pb @ h_rad.T @ np.linalg.inv(R + h_rad @ Pb @ h_rad.T))
    A = K @ h_rad

    #xb = 260
    xb = (np.ones(len(z))*xb).reshape(-1,1)

    xa = xb + K @ (y_rad - h_rad @ xb)

    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(131)

    ax.plot(xa,z,label='Analysis',ls='-',lw=2)
    ax.plot(xb,z,label='Prior',ls='-')
    ax.plot(x_rad,z,c='k',lw=1.5,ls='-',label='True')

    ax.legend(prop=dict(size=14))

    ax.set_xlabel('$T$ (K)',size=20)
    ax.set_ylabel('$z$ (km)',size=20)

    ax.set_title(f'Prior = Constant $X_b$\n$e_R$ = {ep:0.0f}% \t $\sigma_b$ = {sigmab:0.0f} K')
    ax.grid(lw=0.2,axis='y')
    ax.set_xlim(180,300)


    ax = fig.add_subplot(132)
    ax.plot(xa-x_rad,z,label='Analysis',ls='-',lw=2)
    ax.plot(xb-x_rad,z,label='Prior',ls='-')

    ax.legend(prop=dict(size=14))

    ax.set_xlabel('$T$ bias (K)',size=20)
    ax.set_title(f'Prior = Constant $X_b$\n$e_R$ = {ep:0.0f}% \t $\sigma_b$ = {sigmab:0.0f} K')
    ax.grid(lw=0.2,axis='y')
    ax.set_xlim(-50,50)
    ax.axvline(0,c='k',lw=0.3, ls='--')


    ax = fig.add_subplot(133)
    colors = plt.cm.turbo(np.linspace(0,1,10))
    for i in np.arange(0,10,1):
        ii = int(i * 5)
        ax.plot(A[ii,:],z,
                c=colors[i],label=ii+1,lw=1)
    ax.legend(prop=dict(size=12))
    ax.grid(lw=0.2,axis='y')
    ax.set_xlabel('Averaging Kernel', size=20)

    #print(Pb[0,0])

interact(plot_T, 
         ep = FloatSlider(min=1,max=99,step=1,value=5),
         sigmab=FloatSlider(min=1, max=100, step=1, value=50), 
         L=FloatSlider(min=1,step=1,max=50,value=3),
         xb = FloatSlider(min=190,max=290,step=2,value=260),)


interactive(children=(FloatSlider(value=5.0, description='ep', max=99.0, min=1.0, step=1.0), FloatSlider(value…

<function __main__.plot_T(ep, sigmab, L, xb)>

In [20]:
def plot_T(ep,sigmab,L,xbn=1):
    #sigmab = 50
    #L = 3
    
    ep1 = ep*y_rad/100
    R = np.diag(ep1.squeeze()**2)

    Pb = np.full((51,51),0)

    for i in range(51):
        Pb[i,i] = sigmab**2
        #print(i,i,end=' , ')
        for j in range(i):
            #print(i,j)
            Pb[i,j] = (Pb[i,i]*Pb[j,j])**0.5 * np.exp(-((z[i] - z[j])**2/(L**2)))
            #Pb[i,j] = sigmab**2 * np.exp(-((z[i] - z[j])**2/(L**2)))
            Pb[j,i] = Pb[i,j]
        
    K = (Pb @ h_rad.T @ np.linalg.inv(R + h_rad @ Pb @ h_rad.T))
    A = K @ h_rad

    #xb = 260
    #xb = xb_rad1

    xb = xb_rad1
    
    if xbn ==1:
        xb = xb_rad1
    elif xbn == 2:
        xb = xb_rad2

    xa = xb + K @ (y_rad - h_rad @ xb)

    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(131)

    ax.plot(xa,z,label='Analysis',ls='-',lw=2)
    ax.plot(xb,z,label='Prior',ls='-')
    ax.plot(x_rad,z,c='k',lw=1.5,ls='-',label='True')

    ax.legend(prop=dict(size=14))

    ax.set_xlabel('$T$ (K)',size=20)
    ax.set_ylabel('$z$ (km)',size=20)

    ax.set_title(f'Prior = $Xb_{xbn}$\n$e_R$ = {ep:0.0f}% \t $\sigma_b$ = {sigmab:0.0f} K')
    ax.grid(lw=0.2,axis='y')
    ax.set_xlim(180,300)


    ax = fig.add_subplot(132)
    ax.plot(xa-x_rad,z,label='Analysis',ls='-',lw=2)
    ax.plot(xb-x_rad,z,label='Prior',ls='-')

    ax.legend(prop=dict(size=14))

    ax.set_xlabel('$T$ bias (K)',size=20)
    ax.set_title(f'Prior = $Xb_{xbn}$\n$e_R$ = {ep:0.0f}% \t $\sigma_b$ = {sigmab:0.0f} K')
    ax.grid(lw=0.2,axis='y')
    ax.set_xlim(-50,50)
    ax.axvline(0,c='k',lw=0.3, ls='--')
    #ax.set_xlim(180,300)


    ax = fig.add_subplot(133)
    colors = plt.cm.turbo(np.linspace(0,1,10))
    for i in np.arange(0,10,1):
        ii = int(i * 5)
        ax.plot(A[ii,:],z,
                c=colors[i],label=ii+1,lw=1)
    ax.legend(prop=dict(size=12))
    ax.grid(lw=0.2,axis='y')
    ax.set_xlabel('Averaging Kernel', size=20)


    #print(Pb[0,0])

interact(plot_T,ep = FloatSlider(min=1,max=99,step=1,value=5),
          sigmab=FloatSlider(min=1, max=100, step=1, value=50), L=FloatSlider(min=1,step=1,max=50,value=3),
          xbn = RadioButtons(options=[1,2], description='Xb')
)

interactive(children=(FloatSlider(value=5.0, description='ep', max=99.0, min=1.0, step=1.0), FloatSlider(value…

<function __main__.plot_T(ep, sigmab, L, xbn=1)>